
# **SSH File Transfer Protocol (SFTP)**

### <span style="color: green;">Introduction to SFTP</span>

**SSH File Transfer Protocol** or **SFTP** is a protocol designed by the **Internet Engineering Task Force (IETF)** as an extension of the **Secure Shell (SSH)** protocol.

That’s right, **SFTP stands for SSH File Transfer Protocol, and not Secure File Transfer Protocol**, as it is often mistakenly thought to be.

**SSH** in turn stands for the **Secure SHell** protocol. But you may be thinking, **“Isn’t that something else?”**

You are most likely familiar with SSH as a secure method of achieving connectivity to a remote command line terminal, such as to the CLI of a Cisco device or a Linux server.

Although SSH was indeed initially designed as a secure replacement for the Telnet terminal protocol, providing secure login and remote terminal services, it has since evolved into a **cryptographic network protocol** for operating a wide variety of network services securely over an unsecured network.

So, **SFTP leverages the security provided by SSH** to deliver file access, file transfer, and file management services over a network with the same level of security and confidentiality that we have come to expect from SSH remote terminal operations.

---
### <span style="color: green;">SFTP Default Port</span>

**SFTP by default uses port 22**, the same port that SSH uses. This is because SFTP is essentially an extension of the SSH protocol, leveraging its secure channel to transfer files, as mentioned above.

This protocol **encrypts both the commands and the data**, preventing passwords and sensitive information from being transmitted in the clear over the network. This makes it a **secure alternative** to older protocols like FTP (File Transfer Protocol), which transmit data unencrypted. Please have in mind that if the default SSH port 22 is changed on the server side, then SFTP will work fine with the changed port number (e.g 2222 for example).

---

### <span style="color: green;">What is SFTP and How Does it Work?</span>

**SFTP** is a protocol that adheres to the client-server model and goes through the following stages to establish a secure connection, fulfill the file transfer and management process, and then tear down the connection.

---

#### <span style="color: blue;">Establishing a Secure Connection</span>

An SFTP client begins by requesting a secure connection to the SFTP server. This is achieved using standard SSH security procedures. Here’s a brief summary of the steps:

1. **Client Initiates Connection**:
   - The client sends a connection request to the SFTP server, which by default is on port 22 (the default SSH port).

2. **Server Authentication**:
   - The server responds by sending its public key to the client. This allows the client to verify the server’s authenticity, ensuring it’s connecting to the right server and not to a device masquerading as the server.

   If you have used terminal programs like PuTTY with SSH, you might have seen something like this when connecting for the first time:
   <div style="text-align: center;">
     <img src="putty1.png" alt="PuTTY SSH Authentication">
   </div>
   This “ssh-rsa” key is the public key sent by the server to authenticate itself. Every subsequent connection will use that key to check the server's authenticity.

3. **Client Authentication**:
   - Once the server is authenticated, the client is authenticated. This can be done using either:
     - **Password Authentication**: The client sends a password to authenticate itself.
     - **Public Key Authentication**: The client uses a private key to sign data that the server verifies with the corresponding public key.

4. **SSH Session Setup**:
   - After successful authentication, a secure SSH channel is established using symmetric encryption. This encrypted tunnel protects the confidentiality and integrity of data transferred over the connection.

---

#### <span style="color: blue;">File Transfer Process</span>

Once the secure encrypted tunnel has been established, the SFTP subsystem is initiated by the client. This allows the client to issue SFTP commands over the secure SSH tunnel.

- **Common Commands**:
  - **put** and **get**: Used to send and receive files.
  - **Linux-like Commands**: Used to list files, create directories, and change file permissions.

It’s important to note that all commands and files transmitted are sent over the secure SSH tunnel.

---

#### <span style="color: blue;">Session Management (Ending the Session)</span>

Once all intended transactions are complete, the session can be torn down.

---


# **Connecting to SFTP Server Using Python**
Retrieve credentials from the environment file.

In [49]:
import os
from dotenv import load_dotenv
load_dotenv()
host = os.getenv('SFTP_HOST')
username = os.getenv('SFTP_USERNAME')
password = os.getenv('SFTP_PASSWORD')

---

Create an instance of the `SSHClient` class from the `paramiko` library. This client is used to establish an SSH connection to a remote server.

In [22]:
import paramiko as pmk
ssh = pmk.SSHClient()

---

Set the policy for automatically adding the host key of the remote server to the list of known hosts. `AutoAddPolicy` is a policy for automatically trusting the remote server’s SSH key without prompting the user. **This is useful for automation but should be used with caution in production environments due to security risks**.

In [23]:
ssh.set_missing_host_key_policy(pmk.AutoAddPolicy())

---

Establish an SSH connection to the remote server using the provided hostname, username, and password. The connect method handles the authentication process.

In [24]:
ssh.connect(hostname=host, username=username, password=password)

Open an SFTP session over the established SSH connection. The `open_sftp` method returns an `SFTPClient` object, which can be used to perform file operations on the remote server, such as uploading, downloading, and listing files.

In [25]:
sftp_client = ssh.open_sftp()

---
List all methods associated with `sftp_client`

In [28]:
print(dir(sftp_client))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adjust_cwd', '_async_request', '_convert_status', '_cwd', '_expecting', '_finish_responses', '_lock', '_log', '_read_all', '_read_packet', '_read_response', '_request', '_send_packet', '_send_server_version', '_send_version', '_transfer_with_callback', '_write_all', 'chdir', 'chmod', 'chown', 'close', 'file', 'from_transport', 'get', 'get_channel', 'getcwd', 'getfo', 'listdir', 'listdir_attr', 'listdir_iter', 'logger', 'lstat', 'mkdir', 'normalize', 'open', 'posix_rename', 'put', 'putfo', 'readlink', 'remove', 'rename', 'request_number', 'rmdir', 'sock', 'stat', 'symlink', 'truncate', 'ultra_debug', 'unlink', 'utime']


---
Download a file say, `wmr0881.csv` from the remote server to the local machine as `test.csv`

In [18]:
sftp_client.get('/data/WMR_Spots/UK-4pm-wmr/history/wmr0811.csv', 'test.csv')

---

Now let's put all the steps together into a function. Write a function to connect to SFTP server given the credentials.

In [54]:
import paramiko as pmk
def sftp_connect(host=host, username=username, password=password):
    try:
        ssh = pmk.SSHClient()
        ssh.set_missing_host_key_policy(pmk.AutoAddPolicy())
        ssh.connect(hostname=host,
                    username=username,
                    password=password)
        sftp_client = ssh.open_sftp()
        return sftp_client
    except Exception as e:
        print(f"Connection failed: {e}")
        return None

In [56]:
sftp = sftp_connect()

---

Write a function that takes an sftp instance, and downloads `filename` from `remote_dir` inside `local_dir`.

In [62]:
def search_and_download(sftp, remote_dir, filename, local_dir='./'):
    """
    Recursively searches for a specified file in a remote directory and downloads it to a local directory.

    Args:
        sftp (paramiko.SFTPClient): An active SFTP client connection.
        remote_dir (str): The remote directory to start the search from.
        filename (str): The name of the file to search for.
        local_dir (str): The local directory to download the file to.

    Returns:
        bool: True if the file is found and downloaded, False otherwise.

    Raises:
        FileNotFoundError: If the specified file or directory is not found.
        PermissionError: If there are permission issues accessing the file or directory.
        Exception: For any other unexpected errors during the search and download process.

    """
    try:
        print(f"Searching in directory: {remote_dir}")
        for entry in sftp.listdir_attr(remote_dir):
            remote_path = os.path.join(remote_dir, entry.filename).replace('\\', '/')
            if stat.S_ISDIR(entry.st_mode):
                print(f"Entering directory: {remote_path}")
                if search_and_download(sftp, remote_path, filename, local_dir):
                    return True
                    
            elif entry.filename == filename:
                local_path = os.path.join(local_dir, filename)
                print(f"File found: {remote_path}")
                sftp.get(remote_path, local_path)
                print(f"File {filename} downloaded to {local_path}")
                return True
        return False
    except FileNotFoundError as e:
        print(f"Error during search and download: {e}")
        return False
    except PermissionError as e:
        print(f"Permission error: {e}")
        return False
    except Exception as e:
        print(f"Unexpected error: {e}")
        return False

In [63]:
remote_dir = '/'
filename = 'wmr1006.csv'
local_dir = './'

sftp = sftp_connect(host, username, password)
if sftp:
    found = search_and_download(sftp, remote_dir, filename, local_dir)
    if not found:
        print(f"File {filename} not found on the server.")
    sftp.close()

Searching in directory: /
Entering directory: /data
Searching in directory: /data
Entering directory: /data/WMR_Spots
Searching in directory: /data/WMR_Spots
Entering directory: /data/WMR_Spots/CET-2pm
Searching in directory: /data/WMR_Spots/CET-2pm
Entering directory: /data/WMR_Spots/CET-2pm/history
Searching in directory: /data/WMR_Spots/CET-2pm/history
Entering directory: /data/WMR_Spots/UK-4pm-eur
Searching in directory: /data/WMR_Spots/UK-4pm-eur
Entering directory: /data/WMR_Spots/UK-4pm-eur/history
Searching in directory: /data/WMR_Spots/UK-4pm-eur/history
Entering directory: /data/WMR_Spots/UK-4pm-wmr
Searching in directory: /data/WMR_Spots/UK-4pm-wmr
Entering directory: /data/WMR_Spots/UK-4pm-wmr/history
Searching in directory: /data/WMR_Spots/UK-4pm-wmr/history
Entering directory: /data/WMR_Forwards_Outright
Searching in directory: /data/WMR_Forwards_Outright
Entering directory: /data/WMR_Forwards_Outright/UK-4pm-EUR
Searching in directory: /data/WMR_Forwards_Outright/UK-4pm-

---

Now let's write a funcion that uploads a local file to remote server.

In [64]:
def upload_file(sftp, local_file_path, remote_file_path):
    """
    Uploads a local file to a specified remote server path.

    Args:
        sftp (paramiko.SFTPClient): An active SFTP client connection.
        local_file_path (str): The path to the local file to be uploaded.
        remote_file_path (str): The path on the remote server where the file will be uploaded.

    Returns:
        bool: True if the file is successfully uploaded, False otherwise.

    Raises:
        FileNotFoundError: If the local file does not exist.
        Exception: For any other unexpected errors during the upload process.
    """
    try:
        if not os.path.isfile(local_file_path):
            raise FileNotFoundError(f"Local file {local_file_path} does not exist.")
        
        sftp.put(local_file_path, remote_file_path)
        print(f"File {local_file_path} uploaded to {remote_file_path}")
        return True
    except FileNotFoundError as e:
        print(f"Error during upload: {e}")
        return False
    except Exception as e:
        print(f"Unexpected error: {e}")
        return False